In [10]:
import numpy as np
import pandas as pd
from joblib import load

import sys
sys.path.append("C:/Users/Ibrom/Studium/DLMDWME01/Modellentwicklung/")  # Pfad zum Verzeichnis, das die custom_transformers.py Datei enthält
from custom_transformers import extract_time_features, TimeFeaturesExtractor

In [11]:
# Laden der Excel-Datei in ein Pandas Dataframe
df_maintest = pd.read_excel('C:/Users/Ibrom/Studium/DLMDWME01/PSP_main_test.xlsx', sheet_name='Sheet2')
df_maintest

,Unnamed: 0,Id,tmsp,country,amount,success,3D_secured,card
0,NaN,0,2023-12-01 00:01:11,Germany,89,0,0,Visa
1,1.0,1,2023-12-01 00:01:17,Germany,89,1,0,Visa
2,2.0,2,2023-12-01 00:02:49,Germany,238,0,1,Diners
3,3.0,3,2023-12-01 00:03:13,Germany,238,1,1,Diners
4,4.0,4,2023-12-01 00:04:33,Austria,124,0,0,Diners
...,...,...,...,...,...,...,...,...
33346,33346.0,33346,2024-01-07 19:30:10,Germany,121,0,1,Visa
33347,33347.0,33347,2024-01-07 19:32:01,Germany,220,0,0,Master
33348,33348.0,33348,2024-01-07 19:34:16,Austria,332,0,0,Master
33349,33349.0,33349,2024-01-07 19:40:13,Germany,155,0,0,Visa


In [12]:
# Laden der Pipeline
best_pipeline = load('C:/Users/Ibrom/Studium/DLMDWME01/Modellentwicklung/best_pipeline.pkl')

In [13]:
# Extrahieren des OrdinalEncoders für das "card"-Feature aus der Pipeline
ordinal_encoder = best_pipeline.named_steps['preprocessor'].named_transformers_['PSP']

# Ausgabe der Kategorien und ihrer Codierungen für das 'card'-Feature
categories = ordinal_encoder.categories_[0]
print(f"card Kodierungen:")
for index, category in enumerate(categories):
    print(f"{category}: {ordinal_encoder.transform(np.array(category).reshape(-1, 1))[0, 0]}")

card Kodierungen:
Goldcard: 0.0
Moneycard: 1.0
Simplecard: 2.0
UK_Card: 3.0


C:\Users\Ibrom\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
C:\Users\Ibrom\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
C:\Users\Ibrom\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
C:\Users\Ibrom\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


In [14]:
# Kosten für erfolgreiche und fehlgeschlagene Transaktionen für jeden PSP
costs_success = [10, 5, 1, 3]
costs_failure = [5, 2, 0.5, 1]

In [15]:
# Funktion zum Berechnen der Erfolgswahrscheinlichkeiten für jeden PSP
def get_probabilities(transaction, pipeline):
    # Erstellen Sie einen DataFrame für die Transaktion und wiederholen Sie sie für jeden PSP
    data = pd.concat([transaction] * 4, ignore_index=True)
    data['PSP'] = ['UK_Card', 'Simplecard', 'Moneycard', 'Goldcard']

    # Vorhersagen der Erfolgswahrscheinlichkeiten für jeden PSP mit der Pipeline
    probabilities = pipeline.predict_proba(data)
    
    return probabilities[:, 1]  # Nur die Erfolgswahrscheinlichkeiten (Spalte 1) extrahieren

In [16]:
def choose_psp(transaction, pipeline, costs_success, costs_failure):
    probabilities = get_probabilities(transaction, pipeline)
    
    # Schritt 1: Berechnen der erwarteten Versuche für jeden PSP
    expected_attempts = [1 / p for p in probabilities]

    # Schritt 2: Berechnen der erwarteten Kosten für jeden PSP
    costs_expected = [(n - 1) * cost_failure + cost_success for n, cost_success, cost_failure in zip(expected_attempts, costs_success, costs_failure)]

    # Schritt 3: Entscheiden, welcher PSP basierend auf den erwarteten Versuchen und Kosten auszuwählen ist
    psp_below_threshold = [i for i, attempts in enumerate(expected_attempts) if attempts <= 3]

    if psp_below_threshold:
        # Wähle PSP mit minimalen Versuchen unter den PSPs, deren Anzahl an Versuchen <= 3 ist
        chosen_psp = min(psp_below_threshold, key=lambda i: (expected_attempts[i], costs_expected[i]))
    else:
        # Wenn alle PSPs mehr als drei Transaktionsversuche haben, wähle den günstigsten
        chosen_psp = np.argmin(costs_expected)

    return chosen_psp

In [17]:
count_psp = [0, 0, 0, 0]

# Schleife durch alle Transaktionen im DataFrame
for index, transaction in df_maintest.iterrows():
    # Konvertiere die Transaktion in einen DataFrame
    transaction_df = transaction.to_frame().transpose()
    transaction_df.columns = df_maintest.columns

    # Ausgabe der Erfolgswahrscheinlichkeiten für jeden PSP
    probabilities = get_probabilities(transaction_df, best_pipeline)

    # Wählen Sie die beste PSP für die aktuelle Transaktion mit den verschiedenen Funktionen
    chosen_psp = choose_psp(transaction_df, best_pipeline, costs_success, costs_failure)

    # Aktualisieren Sie die Zähler für die ausgewählten PSPs
    count_psp[chosen_psp] += 1

# Ausgabe der Zähler für jeden PSP
print(f"Anzahl der ausgewählten PSPs: {count_psp}")
print(f"Anzahl für Goldcard: {count_psp[0]} \nAnzahl für Moneycard: {count_psp[1]} \nAnzahl für Simplecard: {count_psp[2]} \nAnzahl für UK_Card: {count_psp[3]}")

Anzahl der ausgewählten PSPs: [49, 0, 12352, 20950]
Anzahl für Goldcard: 49 
Anzahl für Moneycard: 0 
Anzahl für Simplecard: 12352 
Anzahl für UK_Card: 20950


In [18]:
print(f"Anzahl für Goldcard: {count_psp[0]} \nAnzahl für Moneycard: {count_psp[1]} \nAnzahl für Simplecard: {count_psp[2]} \nAnzahl für UK_Card: {count_psp[3]}")

Anzahl für Goldcard: 49 
Anzahl für Moneycard: 0 
Anzahl für Simplecard: 12352 
Anzahl für UK_Card: 20950
